# PART 1

**Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.**

In [ ]:
SELECT name
FROM `Facilities`
WHERE membercost >0

**Q2: How many facilities do not charge a fee to members?**

In [ ]:
SELECT COUNT(name)
FROM `Facilities`
WHERE membercost = 0

**Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.**

In [ ]:
SELECT 
  facid,
  name,
  membercost,
  monthlymaintenance,
  membercost/monthlymaintenance AS 'member_maintenance'
FROM  `Facilities` 
WHERE 'member_maintenance' < 0.2 
  AND  membercost > 0.0

**Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.**

In [ ]:
SELECT * 
FROM  `Facilities` 
WHERE  `initialoutlay` <10000
AND  `membercost` >=5
AND  `facid` !=4

**Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.**

In [ ]:
SELECT 
  name,
  monthlymaintenance,
CASE
    WHEN `monthlymaintenance` <= 100 THEN 'cheap'
    WHEN `monthlymaintenance` > 100 THEN 'expensive'
END AS cost_type
FROM `Facilities` 
  ORDER BY cost_type

**Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.**

In [ ]:
SELECT
  surname,
  firstname
FROM Members
ORDER BY `joindate` DESC
LIMIT 1

**Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by
the member name.**

In [ ]:
SELECT DISTINCT CONCAT(m.firstname,' ', m.surname,' ', f.name )
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY m.surname

**Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.**

In [ ]:
SELECT
  Facilities.name as fac_name,
  CONCAT(Members.surname,' ',Members.firstname) AS mem_name,
  CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost*Bookings.slots
       ELSE Facilities.membercost*Bookings.slots END AS Costs
FROM Bookings
LEFT JOIN Facilities
ON Bookings.facid = Facilities.facid
LEFT JOIN Members
ON Bookings.memid = Members.memid
WHERE DATE_FORMAT(Bookings.starttime, '%Y-%m-%d') = '2012-09-14'
HAVING Costs > 30
ORDER BY Costs DESC

**Q9: This time, produce the same result as in Q8, but using a subquery.**

In [ ]:
SELECT 
  CONCAT(Members.surname,' ',Members.firstname) AS mem_name,
  Members.memid,
  sub1.fac_name,
  sub1.Costs
  FROM Members

  RIGHT JOIN(
    SELECT 
    Facilities.name as fac_name,
    Bookings.memid,
    CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost*Bookings.slots
       ELSE Facilities.membercost*Bookings.slots END AS Costs
    FROM Bookings
    LEFT JOIN Facilities
    ON Bookings.facid = Facilities.facid
    WHERE DATE_FORMAT(Bookings.starttime, '%Y-%m-%d') = '2012-09-14'
    HAVING Costs > 30
    ) sub1
  ON sub1.memid = Members.memid
  ORDER BY COSTS DESC

# PART 2

**Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!**

In [ ]:
SELECT facility, SUM(cost) FROM (
    SELECT f.name AS facility, SUM(f.membercost) AS cost
    FROM Facilities AS f
    INNER JOIN Bookings AS b
    ON b.facid = f.facid
    WHERE b.memid != 0
    GROUP BY facility
UNION ALL
    SELECT f.name AS facility, SUM(f.guestcost) AS cost
    FROM Facilities AS f
    INNER JOIN Bookings AS b
    ON b.facid = f.facid
    WHERE b.memid = 0
    GROUP BY facility) as db
WHERE cost < 1000
GROUP BY facility
ORDER BY cost DESC

**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [ ]:
SELECT m.surname, m.firstname, (r.surname || ' ' || r.firstname)
FROM Members AS m
LEFT JOIN Members AS r
ON r.recommendedby = m.memid
ORDER BY m.surname, m.firstname;

**Q12: Find the facilities with their usage by member, but not guests.**

In [ ]:
SELECT f.name AS facility, sub.usage AS usage
FROM Facilities AS f
JOIN (SELECT facid, COUNT(memid) AS usage
FROM Bookings
GROUP BY facid) AS sub
ON f.facid = sub.facid
ORDER by usage DESC

**Q13: Find the facilities usage by month, but not guests.**

In [ ]:
SELECT f.name AS facility, sub.usage AS usage
FROM Facilities AS f
JOIN (SELECT facid, STRFTIME('%m', starttime) AS usage
FROM Bookings
WHERE memid != 0) AS sub
ON f.facid = sub.facid
ORDER by usage DESC